In [31]:
from plotly.offline import init_notebook_mode, iplot

import json
import sys

import plotly.graph_objs as go
import plotly.tools as tools

sys.path.append('..')
from config import root_path


init_notebook_mode(connected=True)

DEFAULT_PLOTLY_COLORS = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                         'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                         'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                         'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                         'rgb(188, 189, 34)', 'rgb(23, 190, 207)']
time_traces = []
languages = list((root_path() / 'src' / 'out' / 'test').glob('**/*.json'))
subplot_titles = [(f.stem + '_accuracy', f.stem + '_loss') for f in languages]
subplot_titles = [name for tup in subplot_titles for name in tup]
fig = tools.make_subplots(rows=len(languages), cols=2, subplot_titles=subplot_titles, print_grid=False)
for idx, file in enumerate(languages):
    with file.open() as f:
        contents = json.load(f)
    n_epochs = contents['Config']['n_epochs']
    metrics = ['train_acc', 'dev_acc', 'loss']
    performance_traces = dict.fromkeys(metrics)
    for trace in performance_traces:
        showlegend = True if idx == 0 else False
        performance_traces[trace] = go.Scatter(
            x=list(range(1, n_epochs)),
            y=[perf[trace] for perf in contents['Performance']],
            mode='lines+markers',
            name=trace,
            line={'color': DEFAULT_PLOTLY_COLORS[metrics.index(trace)]},
            legendgroup=trace,
            showlegend=showlegend
        )
    fig.append_trace(performance_traces['train_acc'], idx+1, 1)
    fig.append_trace(performance_traces['dev_acc'], idx+1, 1)
    fig.append_trace(performance_traces['loss'], idx+1, 2)

buttons = [{'label': 'All',
            'method': 'update',
            'args': [{'visible': [True for i in range(len(subplot_titles))]}, {'title': 'All'}]
            }]
for i, lang in enumerate(languages):
    visibility = [lang.stem == trace.split('_')[0] for trace in subplot_titles]
    button = {'label': lang.stem, 'method': 'update', 'args': [{'visible': visibility}, {'title': lang.stem}]}
    buttons.append(button)
updatemenus = [{'active': -1, 'buttons': buttons}]
fig['layout'].update(height=len(languages)*500, title='All languages performance', updatemenus=updatemenus)
iplot(fig, filename='Languages performance')